# Processing Steps and Discussion
Once we have all of the following data assets assembled, either in a database like I've used in this case, or some collection of files, we can put everything together and build ScienceBase Items.

* Raw data downloaded for cores and cuttings from the CRC Well Catalog application
* Internal IDs tied to "Lib Num" identifiers via harvesting MapServer layers
* Web page info on related artifacts scraped from CRC Well Catalog landing pages for each core/cutting item
* Surficial geology context information retrieved from a Macrostrat API for unique point coordinates

Taking advantage of the fact that, in my case, I have these collections stashed in a local MongoDB instance, I used MongoDB aggregation to pull all of the collections together via their relationships and projected properties that I want to use into a single collection of documents for cores and one for cuttings. This takes care of the grouping on "Lib Num," assembling information from sources, and projection into a standard schema by using $group, $lookup, $project, and $out capabilities of the MongoDB aggregation framework. Because these pipelines are relatively complex and take a while to execute, I ran them via a console connection to MongoDB, but they could just as easily be run via Python using Pymongo. The two pipelines are included in the folder as Javascript files saved from the console app where I built and ran them ([cores_aggregation.js](cores_aggregation.js) and [cuttings_aggregation.js](cuttings_aggregation.js)).

The results of the aggregation output two new final collections, cores and cuttings, containing just the main properties from the whole exercise that I want to use in building ScienceBase Items. For the most part, these are the properties that I can reasonably put into logical parts of the ScienceBase Item model (mainly identifiers, webLinks, contacts, and tags). However, as a reference, I include the full final document construct for each core/cutting in the body as a simple JSON dump, making the content visible for discussion and available for text search through the ScienceBase web app and API.

I broke out all of the logic to assemble the ScienceBase Items from the final cores/cuttings collections into a set of functions in the sbitem_from_crcrecord.py file. This part of the process is kind of ugly and very specific to this exact use case. Other collections would make different choices about what to put where, but there should be some design patterns that could be followed or will at least make for interesting conversation. Ultimately, ScienceBase is not the target pattern to use in designing the model for this kind of information. Rather, there are more direct models available from GeoSciML and other sources that should be examined for a future meta model. In any case, though, some of the key aspects of assigning key concepts to actual qualified vocabularies should be a part of any target model we end up working against. I made some choices here that will set up some of these data elements in a way for the CRC use case that we can talk about further as a community.

I used the ScienceBase capability for batch processing to handle the actual insertion of new records into the two collections. This was done by assembling lists of dictionaries for the two collections in the ScienceBase Item schema, complete with the parentId parameter I injected into the raw data from the tables at the start of my process. This batch process is far superior to any type of looping approach as it allows for a big traunch of data to be sent to the server and then processed there to create the items. I was able to send the entire batch of 52,875 cuttings records in one process and verify at the end that every record inserted by checking to make sure each item in the response had an "id" property (new new ScienceBase Item identifier). (Note: In the codeblocks below, I built the cores and cuttings separately and ran them each through the creation process separately. I only show the final run through the larger cuttings collection as a reference.)

# Dependencies
This notebook uses the Pymongo client installed from Conda-Forge and the sciencebasepy package installed from PyPi. The process relies on a local instance of MongoDB and two collections created via a MongoDB console operation to assemble all final data together for use. I also broke out all of the core functions to build the ScienceBase Item structure into a sbitem_from_crcrecord.py file for readability.

In [1]:
from pymongo import MongoClient
from sciencebasepy import SbSession

from sbitem_from_crcrecord import *

mongo_ndc = MongoClient()

In [2]:
%%time
sb_items_cores = list()
for item in mongo_ndc.crc.cores.find({},{"_id": 0}):
    sb_items_cores.append(sb_item_from_crcwc(item))

CPU times: user 629 ms, sys: 72.7 ms, total: 701 ms
Wall time: 1.03 s


In [3]:
%%time
sb_items_cuttings = list()
for item in mongo_ndc.crc.cuttings.find({},{"_id": 0}):
    sb_items_cuttings.append(sb_item_from_crcwc(item))

CPU times: user 2.91 s, sys: 187 ms, total: 3.1 s
Wall time: 3.92 s


In [4]:
# We have to login to ScienceBase with a user account of creating items under the parentId items referenced in the collections
sb = SbSession()
username = input("Username:  ")
sb.loginc(str(username))

Username:  sbristol@usgs.gov
········


In [5]:
%%time
response = sb.create_items(sb_items_cuttings)

CPU times: user 15.5 s, sys: 10.6 s, total: 26.1 s
Wall time: 1h 21min 35s


In [7]:
# We should get nothing here if every item was inserted successfully; we could also check explicitly for errors in the responses
[i for i in response if "id" not in i.keys()]

[]